In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import random
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import shutil


sys.path.append('../')
from meta_fusion.benchmarks_general import *
from meta_fusion.methods_general import *
from meta_fusion.models_general import *
from meta_fusion.utils_general import *
from meta_fusion.third_party import *
from meta_fusion.config import load_config
from meta_fusion.real_data_general import PrepareRealData

In [3]:
def run_single_experiment(config, extractor_config, random_state, 
                          mod_outs, combined_hiddens, mod_hiddens,
                          is_mod_static, freeze_mod_extractors):

    config['random_state'] = random_state
    extractor_config['random_state'] = random_state
    res_list = []
    best_rho = {}
    cohort_pairs = {}
    ens_idxs={}
    cluster_idxs = {}

    #----------------#
    # Split dataset  #
    #----------------#
    train_loader, val_loader, test_loader = data_preparer.get_data_loaders(random_state=random_state)
    # Get data info
    data_info = data_preparer.get_data_info()
    dim_modalities = data_info[0]
    n = data_info[1]
    n_train = data_info[2]
    n_val = data_info[3]
    n_test = data_info[4]

    print(f"Finished splitting {data_name} dataset. Data information are summarized below:\n"
            f"Modality dimensions: {dim_modalities}\n"
            f"Data size: {n}\n"
            f"Train size: {n_train}\n"
            f"Val size: {n_val}\n"
            f"Test size: {n_test}")
    sys.stdout.flush() 

    #------------------#
    # Benchmark models #
    #------------------#
    bm_extractor = Extractors([[d,0] for d in dim_modalities], dim_modalities, train_loader, val_loader)
    _ = bm_extractor.get_dummy_extractors()
    bm_cohort = Cohorts(extractors=bm_extractor, combined_hidden_layers=combined_hiddens, output_dim=output_dim)

    #----------------------------#
    # Proposed model: MetaJoint  #
    #----------------------------#
    mod_outs = [out+[dim] for out, dim in zip(mod_outs,dim_modalities)] # include the full modalities
    meta_extractor = Extractors(mod_outs, dim_modalities, train_loader, val_loader)
    if (extractor_type == 'encoder') or (extractor_type == 'separate'):
        _ = meta_extractor.get_encoder_extractors(mod_hiddens, separate=separate, config=extractor_config)
    elif extractor_type == 'PCA':
        _ = meta_extractor.get_PCA_extractors()
    meta_cohort = Cohorts(extractors=meta_extractor, combined_hidden_layers=combined_hiddens, output_dim=output_dim,
                          is_mod_static=is_mod_static, freeze_mod_extractors=freeze_mod_extractors)

    #------------------------------#
    #  Train and test benchmarks   #
    #------------------------------#
    bm_models = bm_cohort.get_cohort_models()
    _, bm_dims = bm_cohort.get_cohort_info()
    bm = Benchmarks(config, bm_models, bm_dims, [train_loader, val_loader])
    bm.train()
    res = bm.test(test_loader)
    res_list.append(res)
    print(f"Finished running basic benchmarks!")

    
    #------------------------------#
    #  Train and test MetaJoint   #
    #------------------------------#
    cohort_models = meta_cohort.get_cohort_models()
    _, dim_pairs = meta_cohort.get_cohort_info()
    metafuse = Trainer(config, cohort_models, [train_loader, val_loader])
    metafuse.train() 
    res = metafuse.test(test_loader)
    res_list.append(res)
    
    metafuse.train_ablation() 
    res = metafuse.test_ablation(test_loader)
    res_list.append(res)
    
    best_rho['cohort'] = metafuse.best_rho
    cohort_pairs['cohort'] = dim_pairs
    cohort_pairs['indep_cohort'] = dim_pairs

    if "greedy_ensemble" in config["ensemble_methods"]:
        ens_idxs['greedy_ensemble'] = metafuse.ens_idxs  

    if config['divergence_weight_type'] == "clustering":
        cluster_idxs['cohort'] = metafuse.cluster_idxs

    print(f"Finished running meta fusion!")
    
    results = []
    for res in res_list:
        for method, val in res.items():
            results.append({'Method': method, 'Test_metric': val, 
                            'best_rho':best_rho.get(method), 'cohort_pairs':cohort_pairs.get(method),
                            'ensemble_idxs': ens_idxs.get(method), 'cluster_idxs': cluster_idxs.get(method)})
    

    results = pd.DataFrame(results)

    results['random_state']=random_state
    results["dim_modalities"] = [dim_modalities] * len(results)
    results['n'] = n
    results['n_train'] = n_train
    results['n_val'] = n_val
    results['n_test'] = n_test 

    return results

In [6]:
seed=1
extractor_type = 'encoder'
fine_grained=True
num_modalities = 4 if fine_grained else 3

mod_outs = [[50],[],[],[50]]
combined_hiddens = [64, 32]
mod_hiddens = [[128]]*num_modalities

data_name = 'NACC'
output_dim = 4

if extractor_type == 'encoder':
    separate=False
    is_mod_static=[False]*num_modalities
    freeze_mod_extractors=[False]*num_modalities
elif extractor_type == "separate":
    separate=True
    is_mod_static=[False]*num_modalities
    freeze_mod_extractors=[False]*num_modalities
elif extractor_type == 'PCA':
    separate=False
    is_mod_static=[True]*num_modalities
    freeze_mod_extractors=[False]*num_modalities

config = load_config('../experiments_real/config.json')
extractor_config = load_config('../experiments_real/config_extractor.json')

# Model files directory
ckpt_dir = f"./checkpoints/{data_name}/{extractor_type}_seed{seed}/"
config['ckpt_dir'] = extractor_config['ckpt_dir'] = ckpt_dir

# Update other training parameters
config['divergence_weight_type'] = 'clustering'
config['optimal_k'] = None
config['output_dim'] = extractor_config['output_dim'] = output_dim
config["init_lr"] = 0.001
#config["epochs"] = 20
config["epochs"] = 2
config["ensemble_methods"] = [
        "simple_average",
        "weighted_average",
        "meta_learner",
        "greedy_ensemble"
        ]
extractor_config["init_lr"] = 0.001
#extractor_config["epoch"] = 20
extractor_config["epoch"] = 2

#####################
#    Load Dataset   #
#####################
data_preparer = PrepareRealData(data_name = data_name, test_size = 0.25, val_size = 0.25, fine_grained=fine_grained)
print(f"Finished loading {data_name} dataset.")
sys.stdout.flush() 

Finished loading NACC dataset.


In [7]:
repetition = 1
results = []

for i in tqdm(range(1, repetition+1), desc="Repetitions", leave=True, position=0):
    print(f'Running with repetition {i}...')
    random_state = repetition * (seed-1) + i
    set_random_seed(random_state)
    
    # Run experiment
    tmp = run_single_experiment(config, extractor_config, random_state, mod_outs, combined_hiddens, mod_hiddens,
                          is_mod_static, freeze_mod_extractors)
    
    results.append(tmp)

results = pd.concat(results, ignore_index=True)

Repetitions:   0%|                                                                               | 0/1 [00:00<?, ?it/s]

Running with repetition 1...


C:\Users\liang\anaconda3\envs\fusion\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Finished splitting NACC dataset. Data information are summarized below:
Modality dimensions: [104, 39, 28, 192]
Data size: 1426
Train size: 801
Val size: 268
Test size: 357



100%|███████████████████████████████████████████████████████████████████████████████| 801/801 [00:02<00:00, 330.50it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 801/801 [00:02<00:00, 301.58it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 801/801 [00:02<00:00, 294.57it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 801/801 [00:02<00:00, 277.64it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 801/801 [00:03<00:00, 266.97it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 801/801 [00:02<00:00, 286.41it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 801/801 [00:02<00:00, 277.52it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 801/801 [00:03<00:00, 264.08it/s]

100%|██████████████████████████

...

meta_learner: train task loss: 0.567 - val task loss: 0.575



  0%|                                                                                           | 0/13 [00:00<?, ?it/s]
64it [00:00, 296.17it/s]                                                                                               
128it [00:00, 347.85it/s]
192it [00:00, 413.86it/s]
256it [00:00, 419.96it/s]
320it [00:00, 419.86it/s]
384it [00:00, 426.85it/s]
448it [00:01, 457.54it/s]
512it [00:01, 475.83it/s]
576it [00:01, 478.43it/s]
640it [00:01, 482.29it/s]
704it [00:01, 489.54it/s]
801it [00:01, 432.43it/s]


meta_learner: train task loss: 0.592 - val task loss: 0.571



  0%|                                                                                           | 0/13 [00:00<?, ?it/s]
64it [00:00, 406.50it/s]                                                                                               
128it [00:00, 444.53it/s]
192it [00:00, 472.44it/s]
256it [00:00, 509.58it/s]
320it [00:00, 503.55it/s]
384it [00:00, 491.53it/s]
448it [00:00, 497.95it/s]
512it [00:01, 525.58it/s]
576it [00:01, 530.92it/s]
640it [00:01, 516.32it/s]
704it [00:01, 523.27it/s]
801it [00:01, 485.41it/s]
Repetitions: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [02:16<00:00, 136.66s/it]

meta_learner: train task loss: 0.566 - val task loss: 0.672
Done!
Method: (simple_average), Test_Accuracy: 0.6946778711484594
Method: (weighted_average), Test_Accuracy: 0.6946778711484594
Method: (meta_learner), Test_Accuracy: 0.7647058823529411
Method: (greedy_ensemble), Test_Accuracy: 0.6834733893557423
Method: (best_single), Test_Accuracy: 0.7198879551820728
Method: (cohort), Test_Accuracy: [0.7282913165266106, 0.7030812324929971, 0.7198879551820728, 0.6666666666666666]
Finished running meta fusion!


In [10]:
results

,Method,Test_metric,best_rho,cohort_pairs,ensemble_idxs,cluster_idxs,random_state,dim_modalities,n,n_train,n_val,n_test
0,modality_1,0.621849,NaN,None,None,None,1,"[104, 39, 28, 192]",1426,801,268,357
1,modality_2,0.641457,NaN,None,None,None,1,"[104, 39, 28, 192]",1426,801,268,357
2,modality_3,0.593838,NaN,None,None,None,1,"[104, 39, 28, 192]",1426,801,268,357
3,modality_4,0.669468,NaN,None,None,None,1,"[104, 39, 28, 192]",1426,801,268,357
4,early_fusion,0.717087,NaN,None,None,None,1,"[104, 39, 28, 192]",1426,801,268,357
5,late_fusion,0.621849,NaN,None,None,None,1,"[104, 39, 28, 192]",1426,801,268,357
6,simple_average,0.759104,NaN,None,None,None,1,"[104, 39, 28, 192]",1426,801,268,357
7,weighted_average,0.759104,NaN,None,None,None,1,"[104, 39, 28, 192]",1426,801,268,357
8,meta_learner,0.770308,NaN,None,None,None,1,"[104, 39, 28, 192]",1426,801,268,357
9,greedy_ensemble,0.759104,NaN,None,"[2, 1, 3]",None,1,"[104, 39, 28, 192]",1426,801,268,357
